<a href="https://colab.research.google.com/github/kamgne86/Chatbotchrist/blob/main/agentchabot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

charger les dataset

In [ ]:
from google.colab import files

# Télécharger le fichier
uploaded = files.upload()

Saving model_resnet_emotions.pth to model_resnet_emotions.pth


In [ ]:
!unzip -q  data.zip

In [ ]:
!ls

data	  model_resnet_emotions.pth  sample_movies_with_emotions.csv
data.zip  sample_data


In [ ]:
# 1. Installation des bibliothèques (si besoin)
!pip install langchain openai faiss-cpu pillow

In [ ]:
!pip install -U langchain-community

In [ ]:
# 2. Importations
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub  # ou autre LLM local si besoin
from langchain.schema import Document

In [ ]:
poster_dir = "data/img/"

# 3. Chargement des données de films
movies_df = pd.read_csv("sample_movies_with_emotions.csv")  # colonnes: 'title', 'description', 'poster_path'

In [ ]:
movies_df.head(5)

,movie_id,emotions,poster_path
0,14057,joie,14057.jpg
1,14112,romance,14112.jpg
2,14113,neutre,14113.jpg
3,14114,suspense,14114.jpg
4,14120,peur,14120.jpg


In [ ]:
# 4. Création de documents LangChain
documents = []
for _, row in movies_df.iterrows():
    content = f"Titre: {row['movie_id']}\nDescription: {row['emotions']}"
    metadata = {"poster_path": row['poster_path'], "title": row['movie_id']}
    documents.append(Document(page_content=content, metadata=metadata))

In [ ]:

# 5. Embedding avec SentenceTransformer (gratuit)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

<ipython-input-20-7807dddf7f30>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import model_resnet_emotions as model_resnet_emotions

# 6. Chargement du modèle d'embedding local pour le matching
from sentence_transformers import SentenceTransformer

# Modèle local gratuit performant pour correspondance sémantique
model = SentenceTransformer('all-MiniLM-L6-v2')  # peut être remplacé par un modèle CLIP pour matching image + texte

# Fonction de recherche intelligente
def search_movies(query, k=3):
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Obtenir les résultats les plus proches dans l'index FAISS
    results = vectorstore.similarity_search_by_vector(query_embedding.cpu().numpy(), k=k)
    return results


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

emotion_list = ['joie', 'tristesse', 'colère', 'peur', 'surprise', 'neutre', 'suspense', 'romance', 'aventure', 'drame', 'comédie', 'mystère', 'action']  # Exemple

# 1. Define Model Architecture (ResNet50)
model = models.resnet50(pretrained=False)  # Start with a fresh ResNet50
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(emotion_list))  # Adjust the output layer to match the number of classes


# 2. Load Saved State Dictionary
model_path = "model_resnet_emotions.pth"  # Path to your saved model file
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode


# 3. Define Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 4. Preprocess Input Image
image_path = "path/to/your/new/image.jpg"
image = Image.open(image_path).convert("RGB")
image_tensor = transform(image).unsqueeze(0)  # Add a batch dimension

# 5. Make Prediction
with torch.no_grad():
    output = model(image_tensor)
    _, predicted_idx = torch.max(output, 1)  # Get the index of the predicted class
    predicted_emotion = emotion_list[predicted_idx]

print(f"Predicted Emotion: {predicted_emotion}")

In [ ]:
# 7. Affichage d'affiches
def display_poster(title, poster_path):
    poster_dir = "data/img/"
    path = os.path.join(poster_dir, poster_path)
    if os.path.exists(path):
        image = Image.open(path)
        plt.imshow(image)
        plt.title(title)
        plt.axis("off")
        plt.show()
    else:
        print(f"[⚠] Affiche introuvable pour {title}")

In [ ]:
# 📘 Notebook : Création d'un Agent Conversationnel avec LangChain pour Affiches de Films
# 7. Fonction d'affichage d'affiches

def display_poster(title, poster_path):
    image_path = os.path.join("posters", poster_path)
    if os.path.exists(image_path):
        image = Image.open(image_path)
        plt.imshow(image)
        plt.title(title)
        plt.axis("off")
        plt.show()
    else:
        print(f"Affiche introuvable pour {title}")


In [ ]:
# 8. Interface conversationnelle simple
print("🎬 Chatbot cinéma (gratuit) — Tapez 'exit' pour quitter.")
while True:
    query = input("\nVous : ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Merci, à bientôt !")
        break

    results = search_movies(query)
    print("\n🎥 Suggestions :")
    for doc in results:
        title = doc.metadata["title"]
        poster_path = doc.metadata["poster_path"]
        print(f"- {title}")
        display_poster(title, poster_path)

🎬 Chatbot cinéma (gratuit) — Tapez 'exit' pour quitter.

🎥 Suggestions :
- 192132
Affiche introuvable pour 192132
- 36334
Affiche introuvable pour 36334
- 79316
Affiche introuvable pour 79316
